In [1]:
import tensorflow as tf
import pathlib
import PIL
import numpy as np

import matplotlib
matplotlib.use("TKAgg")
import matplotlib.pyplot as plt

import torch
import torchvision.transforms as transforms
from PIL import Image
import os
import scipy

AUTOTUNE = tf.data.AUTOTUNE # prompts the tf.data runtime to tune the value dynamically at runtime



In [2]:
batch_size = 32
img_height = 28
img_width = 28

def create_dataset_from_path(path):

    # function that converts a file path to an (img, label) pair
    def get_label(file_path):
      # Convert the path to a list of path components
      parts = tf.strings.split(file_path, os.path.sep)
      # The second to last is the class-directory
      one_hot = parts[-2] == class_names
      # Integer encode the label
      return tf.argmax(one_hot)

    def decode_img(img):
      # Convert the compressed string to a 3D uint8 tensor
      img = tf.io.decode_jpeg(img, channels=1)
      # Resize the image to the desired size
      return tf.image.resize(img, [img_height, img_width])

    def process_path(file_path):
      label = get_label(file_path)
      # Load the raw data from the file as a string
      img = tf.io.read_file(file_path)
      img = decode_img(img)
      return img, label

    # Make the imgs in to batches
    def configure_for_performance(ds):
      ds = ds.cache()
      ds = ds.shuffle(buffer_size=1000)
      ds = ds.batch(batch_size)
      ds = ds.prefetch(buffer_size=AUTOTUNE)
      return ds

    data_dir = pathlib.Path(path) # for pycharm
    #data_dir = pathlib.Path('mnist_png/training') # for the jupyter notebook

    print(path+'\\*\\*')

    # Shuffle might be better set to False but this seems better for now
    list_ds = tf.data.Dataset.list_files((path+'\\*\\*'), shuffle=True)

    image_count = len(list_ds)
    class_names = np.array(sorted([item.name for item in data_dir.glob('*')]))
    print(class_names)
    val_size = int(image_count * 0.2)
    train_ds = list_ds.skip(val_size) # skips x initial elements from this dataset.
    val_ds = list_ds.take(val_size)
    print(tf.data.experimental.cardinality(train_ds).numpy())
    print(tf.data.experimental.cardinality(val_ds).numpy())


    # Set `num_parallel_calls` so multiple images are loaded/processed in parallel.
    train_ds = train_ds.map(process_path, num_parallel_calls=AUTOTUNE)
    val_ds = val_ds.map(process_path, num_parallel_calls=AUTOTUNE)

    normalization_layer = tf.keras.layers.Rescaling(1./255)

    normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
    # normalized_ds  -> range goes from 0 to 1
    # train_ds  -> range goes from 0 to

    val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

    normalized_ds = configure_for_performance(normalized_ds)
    val_ds = configure_for_performance(val_ds)

    print(normalized_ds)
    return normalized_ds, val_ds, class_names


def create_dataset_from_img(path):

    def decode_img(img):
      # Convert the compressed string to a 3D uint8 tensor
      img = tf.io.decode_jpeg(img, channels=1)
      # Resize the image to the desired size
      return tf.image.resize(img, [img_height, img_width])

    def process_path(file_path):
      label = tf.strings.split(file_path, os.path.sep)[-2]
      #print(label)
      # Load the raw data from the file as a string
      img = tf.io.read_file(file_path)
      img = decode_img(img)
      return img, label


    # Make the imgs in to batches
    def configure_for_performance(ds):
      ds = ds.cache()
      ds = ds.shuffle(buffer_size=1000)
      ds = ds.batch(1)
      ds = ds.prefetch(buffer_size=AUTOTUNE)
      return ds

    data_dir = pathlib.Path(path) # for pycharm
    #data_dir = pathlib.Path('mnist_png/training') # for the jupyter notebook

    #print(path+'\\*\\*')


    list_ds = tf.data.Dataset.list_files(path, shuffle=True)

    image_count = len(list_ds)
    class_name = [int(tf.strings.split(path, os.path.sep)[-2])]
    #print(f"Class -> {class_name}")
    val_size = int(image_count)
    val_ds = list_ds.take(val_size)
    #print(tf.data.experimental.cardinality(val_ds).numpy())


    # Set `num_parallel_calls` so multiple images are loaded/processed in parallel.
    val_ds = val_ds.map(process_path, num_parallel_calls=AUTOTUNE)

    normalization_layer = tf.keras.layers.Rescaling(1./255)
    val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

    val_ds = configure_for_performance(val_ds)


    return val_ds, class_name

In [5]:
os.getcwd()
os.chdir('D:\AI_Projects\Classifier_AI')
os.getcwd()

'D:\\AI_Projects\\Classifier_AI'

In [8]:
normalized_ds, val_ds, class_names = create_dataset_from_path('mnist_png\\training')

['0' '1' '2' '3' '4' '5' '6' '7' '8' '9']
48000
12000
<PrefetchDataset element_spec=(TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>
mnist_png\training\*\*
['0' '1' '2' '3' '4' '5' '6' '7' '8' '9']
48000
12000
<PrefetchDataset element_spec=(TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>


In [11]:
# Using the normalized
for imageBatch, labelBatch in normalized_ds.take(1):
    print("Image shape: ", imageBatch.numpy()[0].shape)
    print("Label: ", labelBatch.numpy()[0])
    print("Label: ", labelBatch.numpy())
    #print(np.min(image), np.max(image))

    # f = open("demofile.txt", "a")
    # for a in range(28): # goes from 0 to 27
    #   for b in range(28):
    #     print(f'{round(float(image[b][a][0]), 1)}',end='')
    #     f.write(str(round(float(image[b][a][0]), 1)))
    #   print(f'\n')
    #   f.write('\n')
    # f.close()

    imgToBeDisplayed = imageBatch.numpy()[0]

    extraChannel = np.zeros((28,28,1))
    imgToBeDisplayed = np.concatenate((imgToBeDisplayed,extraChannel), axis=2)
    imgToBeDisplayed = np.concatenate((imgToBeDisplayed,extraChannel), axis=2)
    imgToBeDisplayed = imgToBeDisplayed * 255

    print("Image shape: ", imgToBeDisplayed.shape)
    print("Label: ", labelBatch.numpy()[0])
    # #print(f'->{image}')

    print(np.min(imageBatch[0]), np.max(imageBatch[0]))

    image_uint8 = imgToBeDisplayed.astype(np.uint8)
    img = Image.fromarray(image_uint8, 'RGB')
    #img.show()
    #img.save('Classifier_AI\\temp.png')
    img.save('temp.png')

#PIL.Image.open('Classifier_AI\\temp.png')
PIL.Image.open('temp.png')

Image shape:  (28, 28, 1)
Label:  0
Label:  [0 1 3 7 0 4 1 0 5 5 2 6 6 6 3 4 5 6 6 7 2 3 4 0 7 6 9 2 4 8 4 8]
Image shape:  (28, 28, 3)
Label:  0
0.0 1.0


In [12]:
os.remove('temp.png')

In [14]:
num_classes = 10

model = tf.keras.Sequential([
  tf.keras.layers.Conv2D(filters=15, kernel_size=(5,5), padding='Same', activation='relu', input_shape=(28, 28, 1),
                         data_format="channels_last"),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

  tf.keras.layers.Conv2D(filters=20, kernel_size=(10,10), padding='Same', activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

  tf.keras.layers.Conv2D(filters=25, kernel_size=(15,15), padding='Same', activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])

In [15]:
model.compile(
  optimizer='adam',
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])


In [16]:
epochs=10

history = model.fit(
  normalized_ds,
  validation_data=val_ds,
  epochs=epochs
)



Epoch 1/10
1500/1500 [==============================] - 164s 104ms/step - loss: 0.1584 - accuracy: 0.9506 - val_loss: 0.0510 - val_accuracy: 0.9846

In [17]:
model.save('saved_models')

model.summary()

INFO:tensorflow:Assets written to: saved_models\assets


INFO:tensorflow:Assets written to: saved_models\assets


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 28, 28, 15)        390       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 14, 14, 15)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 14, 14, 20)        30020     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 7, 7, 20)         0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 7, 7, 25)          112525    
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 3, 3, 25)        

In [18]:
model = tf.keras.models.load_model('saved_models')

In [20]:
#tf.keras.utils.plot_model(model, to_file='model.png',
#                          show_shapes=True, show_layer_activations=True)

In [21]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)



In [22]:
plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)

plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [23]:
def test_model(img_path):

    test_val_ds, test_class_names = create_dataset_from_img(img_path)

    predictions = model.predict(test_val_ds)
    score = tf.nn.softmax(predictions[0])

    answerBool = (int(class_names[np.argmax(score)]) == test_class_names[0])

    print(
        "[{}] This image most likely belongs to {} with a {:.2f} percent confidence."
        .format(answerBool, class_names[np.argmax(score)], 100 * np.max(score))
    )

    for imageBatch, labelBatch in test_val_ds.take(1):
        imgToBeDisplayed = imageBatch.numpy()[0]

        extraChannel = np.zeros((28,28,1))
        imgToBeDisplayed = np.concatenate((imgToBeDisplayed,extraChannel), axis=2)
        imgToBeDisplayed = np.concatenate((imgToBeDisplayed,extraChannel), axis=2)
        imgToBeDisplayed = imgToBeDisplayed * 255

        image_uint8 = imgToBeDisplayed.astype(np.uint8)
        img = Image.fromarray(image_uint8, 'RGB')
        #img.show()
        img.save('temp.png')


In [24]:
test_model('D:\\AI_Projects\\Classifier_AI\\mnist_png\\testing\\6\\156.png')
PIL.Image.open('temp.png')

1/1 [==============================] - 0s 374ms/step
[True] This image most likely belongs to 6 with a 100.00 percent confidence.


In [27]:
convert_tensor = transforms.ToTensor()

img_tensor = convert_tensor(img)

print(img_tensor.shape)
testing_ds = tf.keras.utils.image_dataset_from_directory(
    pathlib.Path('mnist_png/testing'),
    seed=123,
    image_size=(img_height, img_width),
    batch_size=1,
    color_mode='grayscale'
)

torch.Size([1, 28, 28])
Found 10000 files belonging to 10 classes.


In [28]:
model.predict(testing_ds)

10000/10000 [==============================] - 72s 7ms/step

array([[-3713.414  , -2953.0027 , 10585.379  , ..., -2495.7432 ,
        -2003.1846 , -6735.457  ],
       [-1496.1576 , -1823.1067 ,  5445.316  , ..., -1144.4215 ,
          548.9529 , -3476.1921 ],
       [-1348.5408 , -1436.3109 ,  -116.94449, ...,  5045.144  ,
        -2302.782  , -1539.2506 ],
       ...,
       [ -113.28211,  -952.22723,    84.39344, ...,  1354.7083 ,
          -97.05953,  -284.47232],
       [ 5279.7007 , -2969.476  , -2732.6067 , ..., -2636.6152 ,
        -1950.6704 ,   848.08105],
       [-2604.307  ,  -315.16455, -1984.3174 , ...,  -298.81985,
        -1793.1935 , -1739.9471 ]], dtype=float32)